In [ ]:
# part of the notebook is adapted from A. Zonca https://nbviewer.org/gist/zonca/6187504

In [ ]:
import numpy as np
import healpy as hp
from scipy.interpolate import interp1d

import matplotlib.pyplot as plt

from PIL import Image
from matplotlib.image import pil_to_array

In [ ]:
%matplotlib inline

In [ ]:
n_side = 2
n_pix = hp.nside2npix(n_side)
approx_res_in_arcmin = hp.nside2resol(n_side, arcmin=True)

print("n_side = %i\nn_pixel = %i\napproximate resolution = %f arcmin" % (n_side, n_pix, approx_res_in_arcmin))

values_of_pixels = np.arange(n_pix)

hp.mollview(values_of_pixels, title="Healpix pixellization")
plt.show()

In [ ]:
n_side = 512
n_pix = hp.nside2npix(n_side)
approx_res_in_arcmin = hp.nside2resol(n_side, arcmin=True)

print("n_side = %i\nn_pixel = %i\napproximate resolution = %f arcmin" % (n_side, n_pix, approx_res_in_arcmin))

Asiago_position = hp.ang2vec(np.radians(44), np.radians(12))
large_ipix_disc = hp.query_disc(nside=n_side, vec=Asiago_position, radius=np.radians(7))
small_ipix_disc = hp.query_disc(nside=n_side, vec=Asiago_position, radius=np.radians(5))
ipix_disc = res = list(set(large_ipix_disc)^set(small_ipix_disc))

# Earth image extracted from basemap:
# https://github.com/matplotlib/basemap/blob/master/lib/mpl_toolkits/basemap/data/shadedrelief.jpg
grayscale_pil_image = Image.open("./shadedrelief.jpg").convert("L")
image_array = pil_to_array(grayscale_pil_image)

theta = np.linspace(0, np.pi, num=image_array.shape[0])[:, None]
phi = np.linspace(-np.pi, np.pi, num=image_array.shape[1])
pix = hp.ang2pix(n_side, theta, phi)



values_of_pixels = np.zeros(hp.nside2npix(n_side), dtype=np.double)
values_of_pixels[pix] = image_array

values_of_pixels[ipix_disc] = values_of_pixels.max()*1.1

hp.mollview(values_of_pixels, title="Position of Asiago", flip="geo")
plt.show()

values_of_pixels_smoothed = hp.smoothing(values_of_pixels, fwhm=np.radians(10.))

hp.mollview(values_of_pixels_smoothed, title="Same but smoothed", flip="geo")
plt.show()

hp.gnomview(values_of_pixels, rot=(12.5, 41.9), reso=.5, xsize=1600, title="Gnomonic projection of Italy", flip="geo")
plt.show()

In [ ]:
cl = hp.anafast(values_of_pixels, lmax=1024)
cl_smoothed = hp.anafast(values_of_pixels_smoothed, lmax=1024)
ell = np.arange(len(cl))

In [ ]:
plt.plot(ell, ell * (ell + 1) * cl/(2.*np.pi), label="raw")
plt.plot(ell, ell * (ell + 1) * cl_smoothed/(2.*np.pi), label="smoothed")
plt.xlabel("$\ell$")
plt.ylabel("$\ell(\ell+1)C_{\ell}/(2\pi)$")
plt.grid()
plt.legend(frameon=False)
plt.loglog()
plt.xlim(0,1000)
plt.ylim(1., 1.e3)
hp.write_cl("cl.fits", cl, overwrite=True)